# Bash - Commands used for running the analyses in the Workshop

## Exploring the Genotype Data

In [12]:
SDIR=~/share;
cd $SDIR/genotype_data;
ls

genotypes.geno  genotypes.ind  genotypes.snp  outgroupf3stats.params.txt


Exploring the files. Here are the first 20 individuals:

In [2]:
head -20 genotypes.ind

             Yuk_009 M    Yukagir
             Yuk_025 F    Yukagir
             Yuk_022 F    Yukagir
             Yuk_020 F    Yukagir
               MC_40 M    Chukchi
             Yuk_024 F    Yukagir
             Yuk_023 F    Yukagir
               MC_16 M    Chukchi
               MC_15 F    Chukchi
               MC_18 M    Chukchi
             Yuk_004 M    Yukagir
               MC_08 F    Chukchi
             Nov_005 M   Nganasan
               MC_25 F    Chukchi
             Yuk_019 F    Yukagir
             Yuk_011 M    Yukagir
             Sesk_47 M   Chukchi1
               MC_17 M    Chukchi
             Yuk_021 M    Yukagir
               MC_06 F    Chukchi


And here the first 20 SNP rows:

In [3]:
head -20 genotypes.snp

            1_752566     1        0.020130          752566 G A
            1_842013     1        0.022518          842013 T G
            1_891021     1        0.024116          891021 G A
            1_903426     1        0.024457          903426 C T
            1_949654     1        0.025727          949654 A G
           1_1018704     1        0.026288         1018704 A G
           1_1045331     1        0.026665         1045331 G A
           1_1048955     1        0.026674         1048955 A G
           1_1061166     1        0.026711         1061166 T C
           1_1108637     1        0.028311         1108637 G A
           1_1120431     1        0.028916         1120431 G A
           1_1156131     1        0.029335         1156131 T C
           1_1157547     1        0.029356         1157547 T C
           1_1158277     1        0.029367         1158277 G A
           1_1161780     1        0.029391         1161780 C T
           1_1170587     1        0.029450         1170

And here are the first 20 genotypes of the first 100 individuals:

In [4]:
head -20 genotypes.geno | cut -c1-100

0101101211102210102021200100010200000011001000200001010110001100001111101001110200110111212110000011
2012121012210011122100111202201222121102222121121012202221211212202201101201220222122012011112122022
1100112001110021001001111000011200000111100001110001110100002100110111120000102200110112011020001001
0000112210222121221121100202221222122112112211202122222221022222111221102200112222122211012121022011
0000000000000000000000000000100000000000000000100010000000000000000000000000000100000011201120010000
1012100221102201101121110120110000010012002010200100010011100100011011101110120200010100000002010111
2222222222222222222222222222222222222222222222222222222222222222222222222222222222121222222222122222
2211222002212022102001212222212212222210122212121222112222221112122111222222122021221122222222222222
2211222002212022102001212202012212212210122212121122112221221112121111222122112021211121011121222211
2222222222102222202222222222222222222222222211222212122222122122222222222222222122221212212

Counting how many individuals and SNPs there are:

In [5]:
wc -l genotypes.ind
wc -l genotypes.snp

1351 genotypes.ind
593124 genotypes.snp


And now we check that the first row of the `*.geno` file indeed contains the same number of columns:

In [6]:
head -1 genotypes.geno | wc -c

1352


which is one more, including the newline character at the end of the line. Now counting the number of rows in the `*.geno`-file (this takes a few seconds, as the file is >2Gb):

In [7]:
wc -l genotypes.geno

593124 genotypes.geno


Great, the number of rows and columns agrees with the numbers indicated in the `*.ind` and `*.snp` file!
Now we're counting how many different populations there are. Let's first see the first 10 populations in the sorted list, alongside the number of individuals in each group:

In [8]:
awk '{print $3}' genotypes.ind | sort | uniq -c | head -20

      9 Abkhasian
     16 Adygei
      6 Albanian
      7 Aleut
      4 Aleut_Tlingit
      7 Altaian
     10 Ami
     10 Armenian
      9 Atayal
     10 Balkar
     29 Basque
     25 BedouinA
     19 BedouinB
     10 Belarusian
      6 BolshoyOleniOstrov
      9 Borneo
     10 Bulgarian
      8 Cambodian
      2 Canary_Islander
      2 ChalmnyVarre


As you can see, there are a number of populations with only one sample. Those are typically ancient individuals which are considered individually in most analyses. Let's filter those out and count only populations with at least two individuals and count them:

In [9]:
awk '{print $3}' genotypes.ind | sort | uniq -c | awk '$1>1' | wc -l

114


OK, so there are 114 populations in this dataset.

## Running `smartPCA`

We first have to create a parameter file with the input and output data.

In [73]:
WDIR=~/work/share/solutions
mkdir -p $WDIR
cd $WDIR
cat <<EOF > pca.WestEurasia.params.txt
genotypename: $SDIR/genotype_data/genotypes.geno
snpname: $SDIR/genotype_data/genotypes.snp
indivname: $SDIR/genotype_data/genotypes.ind
evecoutname: $WDIR/pca.WestEurasia.evec
evaloutname: $WDIR/pca.WestEurasia.eval
poplistname: $SDIR/WestEurasia.poplist.txt
lsqproject: YES
numoutevec: 4
EOF

Let's see whether it worked:

In [74]:
cat pca.WestEurasia.params.txt

genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
evecoutname: /home/training/work/share/solutions/pca.WestEurasia.evec
evaloutname: /home/training/work/share/solutions/pca.WestEurasia.eval
poplistname: /home/training/share/WestEurasia.poplist.txt
lsqproject: YES
numoutevec: 4


Good, now we can run `smartPCA`:

In [18]:
time smartpca -p pca.WestEurasia.params.txt

parameter file: pca.params.txt
### THE INPUT PARAMETERS
##PARAMETER NAME: VALUE
genotypename: /home/training/work/share/genotype_data/genotypes.smaller.geno
snpname: /home/training/work/share/genotype_data/genotypes.smaller.snp
indivname: /home/training/work/share/genotype_data/genotypes.smaller.ind
evecoutname: /home/training/work/share/solutions/pca.WestEurasia.evec
evaloutname: /home/training/work/share/solutions/pca.WestEurasia.eval
poplistname: /home/training/work/share/WestEurasia.poplist.txt
lsqproject: YES
numoutevec: 4
## smartpca version: 16000
norm used

lsqproject used
number of samples used: 771 number of snps used: 593124
Using 3 threads, and partial sum lookup algorithm.
 snp            1_1330931 ignored . allelecnt:     0  missing:     1
 snp            1_1340241 ignored . allelecnt:     0  missing:     0
 snp            1_1611813 ignored . allelecnt:     0  missing:     1
 snp            1_1770429 ignored . allelecnt:     0  missing:     1
 snp            1_1774353 ign

  42      1.397946   -0.001962    39.534  6.02179e-74 24934.102
  43      1.395569   -0.002377    39.769  1.36881e-74 25115.341
  44      1.390360   -0.005210    38.864  4.03152e-72 25298.940
  45      1.385084   -0.005276    37.904  1.54925e-69 25481.309
  46      1.378586   -0.006497    36.420  1.32902e-65 25662.240
  47      1.377115   -0.001472    36.950  5.34186e-67 25840.026
  48      1.374029   -0.003086    36.825  1.14218e-66 26021.045
  49      1.370217   -0.003812    36.392  1.56757e-65 26203.234
  50      1.362597   -0.007620    34.367  2.70897e-60 26385.597
  51      1.360695   -0.001902    34.672  4.52441e-61 26562.928
  52      1.359427   -0.001268    35.243  1.53286e-62 26742.751
  53      1.354377   -0.005050    34.235  5.87917e-60 26925.995
  54      1.350525   -0.003851    33.705  1.29466e-58 27107.447
  55      1.344736   -0.005789    32.338  3.37776e-55 27288.662
  56      1.343233   -0.001503    32.761  3.02252e-56 27466.815
  57      1.339959   -0.003274    32.431

 171      1.095494   -0.002907    -5.866     0.999995 45045.703
 172      1.094875   -0.000618    -5.671     0.999985 45177.436
 173      1.094238   -0.000638    -5.485     0.999965 45310.832
 174      1.093169   -0.001069    -5.553     0.999974 45445.858
 175      1.091754   -0.001415    -5.828     0.999993 45581.141
 176      1.090332   -0.001422    -6.112     0.999999 45715.533
 177      1.089938   -0.000394    -5.789     0.999992 45848.974
 178      1.089230   -0.000708    -5.649     0.999984 45984.829
 179      1.087557   -0.001673    -6.085     0.999998 46122.100
 180      1.087194   -0.000363    -5.743      0.99999 46257.552
 181      1.086377   -0.000817    -5.669     0.999985 46395.570
 182      1.084882   -0.001495    -6.003     0.999997 46534.652
 183      1.084031   -0.000851    -5.955     0.999997 46672.470
 184      1.083202   -0.000829    -5.893     0.999995 46811.206
 185      1.082490   -0.000712    -5.761     0.999991 46950.947
 186      1.081516   -0.000974    -5.786

 300      0.980450   -0.000011   -14.441            1 62265.103
 301      0.978894   -0.001556   -15.075            1 62404.090
 302      0.978277   -0.000616   -15.040            1 62537.082
 303      0.977965   -0.000313   -14.786            1 62670.831
 304      0.976690   -0.001275   -15.225            1 62807.592
 305      0.976150   -0.000539   -15.137            1 62940.256
 306      0.975217   -0.000934   -15.334            1 63074.221
 307      0.974853   -0.000364   -15.121            1 63206.533
 308      0.974546   -0.000307   -14.864            1 63341.460
 309      0.973920   -0.000626   -14.837            1 63479.494
 310      0.972813   -0.001107   -15.160            1 63618.244
 311      0.972662   -0.000151   -14.790            1 63753.978
 312      0.971645   -0.001017   -15.048            1 63894.083
 313      0.970437   -0.001208   -15.450            1 64031.830
 314      0.970208   -0.000229   -15.141            1 64165.643
 315      0.969803   -0.000405   -14.958

 429      0.899792   -0.000135   -15.279            1 81026.453
 430      0.898742   -0.001050   -15.660            1 81204.822
 431      0.897845   -0.000897   -15.919            1 81374.909
 432      0.897437   -0.000407   -15.776            1 81539.348
 433      0.896148   -0.001289   -16.365            1 81707.201
 434      0.896008   -0.000141   -16.006            1 81861.795
 435      0.895563   -0.000445   -15.896            1 82024.671
 436      0.895410   -0.000152   -15.541            1 82190.212
 437      0.894941   -0.000469   -15.445            1 82364.227
 438      0.894237   -0.000704   -15.544            1 82540.720
 439      0.893422   -0.000815   -15.739            1 82715.061
 440      0.892932   -0.000490   -15.664            1 82884.934
 441      0.891691   -0.001241   -16.220            1 83056.753
 442      0.891632   -0.000059   -15.791            1 83215.283
 443      0.891039   -0.000593   -15.805            1 83384.278
 444      0.890625   -0.000414   -15.669

...

popdifference:                 Kumyk              Scottish       110.143   6.78543e-23       8       4
popdifference:                 Kumyk            Shetlandic         0.000             1       8       0
popdifference:                 Kumyk              Sicilian       148.998   3.33707e-31       8      11
popdifference:                 Kumyk                  Sorb         0.000             1       8       0
popdifference:                 Kumyk         Spanish_North       145.694   1.70332e-30       8       5
popdifference:                 Kumyk               Spanish       483.288  2.75611e-103       8      53
popdifference:                 Kumyk                Syrian        69.509   2.88222e-14       8       3
popdifference:                 Kumyk               Turkish        34.586   5.64882e-07       8      50
popdifference:                 Kumyk             Ukrainian       194.849   4.81007e-41       8       9
popdifference:    Lebanese_Christian              Lebanese         0.000 

popdifference:                Lezgin             Mordovian       298.980   1.79783e-63       9      10
popdifference:                Lezgin        North_Ossetian         0.000             1       9       0
popdifference:                Lezgin             Norwegian       262.247    1.4953e-55       9      11
popdifference:                Lezgin              Orcadian       269.057   5.09506e-57       9      13
popdifference:                Lezgin           Palestinian       373.894   1.21347e-79       9      26
popdifference:                Lezgin                Polish         0.000             1       9       0
popdifference:                Lezgin              Romanian         0.000             1       9       0
popdifference:                Lezgin               Russian       457.559   1.00822e-97       9      22
popdifference:                Lezgin             Sardinian       400.621    2.0424e-85       9      27
popdifference:                Lezgin                 Saudi       126.484 

popdifference:        North_Ossetian              Romanian         0.000             1       0       0
popdifference:        North_Ossetian               Russian         0.000             1       0      22
popdifference:        North_Ossetian             Sardinian         0.000             1       0      27
popdifference:        North_Ossetian                 Saudi         0.000             1       0       8
popdifference:        North_Ossetian              Scottish         0.000             1       0       4
popdifference:        North_Ossetian            Shetlandic         0.000             1       0       0
popdifference:        North_Ossetian              Sicilian         0.000             1       0      11
popdifference:        North_Ossetian                  Sorb         0.000             1       0       0
popdifference:        North_Ossetian         Spanish_North         0.000             1       0       5
popdifference:        North_Ossetian               Spanish         0.000 

popdifference:              Romanian                Syrian         0.000             1       0       3
popdifference:              Romanian               Turkish         0.000             1       0      50
popdifference:              Romanian             Ukrainian         0.000             1       0       9
popdifference:               Russian             Sardinian       898.418  3.66869e-193      22      27
popdifference:               Russian                 Saudi       176.242    4.7818e-37      22       8
popdifference:               Russian              Scottish       203.860    5.5571e-43      22       4
popdifference:               Russian            Shetlandic         0.000             1      22       0
popdifference:               Russian              Sicilian       485.189  1.06973e-103      22      11
popdifference:               Russian                  Sorb         0.000             1      22       0
popdifference:               Russian         Spanish_North       352.726 

eigbestsnp    2          2_103409685  2    103409685     4.478
eigbestsnp    2         11_118221349 11    118221349     4.438
eigbestsnp    3           5_74762246  5     74762246     6.963
eigbestsnp    3           5_74776350  5     74776350     6.953
eigbestsnp    3          1_160302298  1    160302298     6.860
eigbestsnp    3          5_136130864  5    136130864     6.835
eigbestsnp    3          5_136515002  5    136515002     6.825
eigbestsnp    3          5_136406267  5    136406267     6.820
eigbestsnp    3           5_74562849  5     74562849     6.631
eigbestsnp    3          8_102141269  8    102141269     6.557
eigbestsnp    3          2_215349037  2    215349037     6.556
eigbestsnp    3            4_4539599  4      4539599     6.497
eigbestsnp    4          3_177312283  3    177312283     9.936
eigbestsnp    4          2_130513024  2    130513024     9.157
eigbestsnp    4          2_138975301  2    138975301     8.662
eigbestsnp    4          4_145668332  4    145668332   

This has run for 15 minutes. Now we'll run the AllEurasia one:

In [75]:
WDIR=~/work/share/solutions
mkdir -p $WDIR
cd $WDIR
cat <<EOF > pca.AllEurasia.params.txt
genotypename: $SDIR/genotype_data/genotypes.geno
snpname: $SDIR/genotype_data/genotypes.snp
indivname: $SDIR/genotype_data/genotypes.ind
evecoutname: $WDIR/pca.AllEurasia.evec
evaloutname: $WDIR/pca.AllEurasia.eval
poplistname: $SDIR/AllEurasia.poplist.txt
lsqproject: YES
numoutevec: 4
EOF

In [76]:
cat pca.AllEurasia.params.txt

genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
evecoutname: /home/training/work/share/solutions/pca.AllEurasia.evec
evaloutname: /home/training/work/share/solutions/pca.AllEurasia.eval
poplistname: /home/training/share/AllEurasia.poplist.txt
lsqproject: YES
numoutevec: 4


In [77]:
time smartpca -p pca.AllEurasia.params.txt

parameter file: pca.AllEurasia.params.txt
### THE INPUT PARAMETERS
##PARAMETER NAME: VALUE
genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
evecoutname: /home/training/work/share/solutions/pca.AllEurasia.evec
evaloutname: /home/training/work/share/solutions/pca.AllEurasia.eval
poplistname: /home/training/share/AllEurasia.poplist.txt
lsqproject: YES
numoutevec: 4
## smartpca version: 16000
norm used

lsqproject used
number of samples used: 1301 number of snps used: 593124
Using 3 threads, and partial sum lookup algorithm.
 snp            1_1330931 ignored . allelecnt:     0  missing:     1
 snp            1_1340241 ignored . allelecnt:     0  missing:     0
 snp            1_1770429 ignored . allelecnt:     0  missing:     2
 snp            1_1774353 ignored . allelecnt:     0  missing:     0
 snp            1_1842017 ignored . allelecnt:     0  missing:  

  52      1.574901   -0.001898    99.686 3.03035e-290 23892.120
  53      1.566180   -0.008720    97.539 5.52978e-281 24049.781
  54      1.559824   -0.006356    96.365 5.80858e-276 24205.411
  55      1.556275   -0.003549    96.383  4.8769e-276 24360.410
  56      1.549500   -0.006774    94.982  4.3821e-270 24516.519
  57      1.542913   -0.006587    93.627 2.27633e-264 24671.641
  58      1.540315   -0.002598    94.000 6.15187e-266 24825.823
  59      1.531804   -0.008511    91.750 1.61552e-256 24981.621
  60      1.528939   -0.002864    91.965 2.06745e-257 25135.121
  61      1.521507   -0.007432    90.132  8.1466e-250 25289.985
  62      1.515997   -0.005510    89.121 1.17189e-245 25443.138
  63      1.512632   -0.003365    89.053 2.24195e-245 25595.794
  64      1.512015   -0.000617    90.222 3.46786e-250 25749.364
  65      1.507849   -0.004166    89.792 2.04728e-248 25905.746
  66      1.499872   -0.007978    87.605 1.81864e-239 26062.520
  67      1.496241   -0.003630    87.366

 181      1.165652   -0.002221    18.150  2.79601e-24 41500.410
 182      1.164344   -0.001308    17.969  6.04927e-24 41611.169
 183      1.163405   -0.000938    18.021  4.86081e-24 41721.979
 184      1.159586   -0.003819    16.236  8.03763e-21 41833.306
 185      1.158722   -0.000865    16.318  5.76128e-21 41941.469
 186      1.158274   -0.000448    16.667  1.39256e-21 42050.191
 187      1.156118   -0.002155    15.924  2.82475e-20 42160.012
 188      1.154081   -0.002037    15.248  4.11038e-19 42268.736
 189      1.152420   -0.001661    14.807  2.29122e-18 42376.492
 190      1.151642   -0.000777    14.930  1.42541e-18 42483.745
 191      1.149088   -0.002554    13.906  7.07867e-17 42591.631
 192      1.147936   -0.001152    13.778   1.1434e-16 42697.828
 193      1.146993   -0.000943    13.782  1.12275e-16 42804.141
 194      1.144313   -0.002681    12.659  6.80926e-15 42910.841
 195      1.141345   -0.002968    11.337  6.80934e-13 43015.630
 196      1.137728   -0.003616     9.580

 310      1.021338   -0.000697   -16.418            1 52515.995
 311      1.020752   -0.000586   -16.441            1 52583.658
 312      1.019220   -0.001532   -17.174            1 52651.406
 313      1.018966   -0.000254   -16.954            1 52717.490
 314      1.018401   -0.000565   -16.966            1 52784.251
 315      1.016882   -0.001519   -17.696            1 52851.118
 316      1.016381   -0.000501   -17.664            1 52916.318
 317      1.016260   -0.000121   -17.347            1 52981.724
 318      1.015723   -0.000537   -17.341            1 53048.049
 319      1.015156   -0.000567   -17.358            1 53114.522
 320      1.013542   -0.001614   -18.166            1 53181.086
 321      1.013257   -0.000285   -17.976            1 53245.763
 322      1.012991   -0.000266   -17.770            1 53311.043
 323      1.011783   -0.001208   -18.276            1 53376.968
 324      1.011264   -0.000520   -18.265            1 53441.750
 325      1.010738   -0.000526   -18.258

 439      0.944962   -0.000444   -26.952            1 60114.576
 440      0.944757   -0.000204   -26.761            1 60165.169
 441      0.944197   -0.000561   -26.863            1 60216.426
 442      0.943876   -0.000320   -26.767            1 60267.404
 443      0.942923   -0.000954   -27.195            1 60318.742
 444      0.942592   -0.000331   -27.110            1 60368.746
 445      0.942162   -0.000430   -27.107            1 60419.070
 446      0.941549   -0.000612   -27.255            1 60469.452
 447      0.941221   -0.000328   -27.168            1 60519.398
 448      0.940900   -0.000321   -27.075            1 60569.671
 449      0.940533   -0.000367   -27.021            1 60620.294
 450      0.939559   -0.000974   -27.470            1 60671.143
 451      0.939265   -0.000294   -27.356            1 60720.562
 452      0.939117   -0.000148   -27.121            1 60770.396
 453      0.938712   -0.000406   -27.099            1 60821.052
 454      0.938331   -0.000380   -27.056

...

popdifference:              Orcadian                   She       461.500   1.41775e-98      13      10
popdifference:              Orcadian             Sherpa.DG       283.690   3.56699e-60      13       2
popdifference:              Orcadian              Sicilian       211.452   1.29442e-44      13      11
popdifference:              Orcadian               Spanish       287.227   6.16077e-61      13      53
popdifference:              Orcadian         Spanish_North        88.474   2.77725e-18      13       5
popdifference:              Orcadian                Syrian       141.134    1.6135e-29      13       5
popdifference:              Orcadian                 Tajik       226.257   8.44136e-48      13       8
popdifference:              Orcadian                  Thai       351.843   7.01556e-75      13       7
popdifference:              Orcadian            Tibetan.DG       288.801   2.82008e-61      13       2
popdifference:              Orcadian                    Tu       438.239 

popdifference:              Ossetian               Turkmen        59.054   4.58363e-12      10       7
popdifference:              Ossetian              Tuvinian       259.620   5.50559e-55      10      10
popdifference:              Ossetian             Ukrainian       216.247   1.20388e-45      10       9
popdifference:              Ossetian                 Ulchi       479.036  2.28927e-102      10      25
popdifference:              Ossetian                 Uygur       116.787   2.59319e-24      10      10
popdifference:              Ossetian                 Uzbek       100.522   7.61452e-21      10      10
popdifference:              Ossetian                  Xibo       228.685   2.53329e-48      10       7
popdifference:              Ossetian                 Yakut       474.531  2.15744e-101      10      20
popdifference:              Ossetian                    Yi       270.029   3.14403e-57      10      10
popdifference:              Ossetian               Yukagir       103.750 

popdifference:               Russian                   She       610.250   9.3714e-131      22      10
popdifference:               Russian             Sherpa.DG       371.792   3.45075e-79      22       2
popdifference:               Russian              Sicilian       402.321   8.76391e-86      22      11
popdifference:               Russian               Spanish       693.549  8.68984e-149      22      53
popdifference:               Russian         Spanish_North       205.849   2.07605e-43      22       5
popdifference:               Russian                Syrian       251.782   2.68965e-53      22       5
popdifference:               Russian                 Tajik       272.970   7.30295e-58      22       8
popdifference:               Russian                  Thai       508.774  8.48062e-109      22       7
popdifference:               Russian            Tibetan.DG       376.535   3.26273e-80      22       2
popdifference:               Russian                    Tu       570.507 

popdifference:             Sardinian                 Saudi       270.563   2.41254e-57      27       8
popdifference:             Sardinian              Scottish       256.733   2.30647e-54      27       4
popdifference:             Sardinian                Selkup       367.459   2.97798e-78      27      10
popdifference:             Sardinian               Semende       747.392  1.90332e-160      27      10
popdifference:             Sardinian                   She       590.916  1.43302e-126      27      10
popdifference:             Sardinian             Sherpa.DG       342.135   8.74826e-73      27       2
popdifference:             Sardinian              Sicilian       157.295    5.5586e-33      27      11
popdifference:             Sardinian               Spanish       358.677   2.34583e-76      27      53
popdifference:             Sardinian         Spanish_North       209.757   2.99746e-44      27       5
popdifference:             Sardinian                Syrian       205.830 

popdifference:                Selkup             Sherpa.DG        50.392   2.99025e-10      10       2
popdifference:                Selkup              Sicilian       184.284    8.9625e-39      10      11
popdifference:                Selkup               Spanish       634.327  5.75703e-136      10      53
popdifference:                Selkup         Spanish_North       125.803    3.0745e-26      10       5
popdifference:                Selkup                Syrian       132.597   1.08396e-27      10       5
popdifference:                Selkup                 Tajik       124.667    5.3768e-26      10       8
popdifference:                Selkup                  Thai       146.353   1.23066e-30      10       7
popdifference:                Selkup            Tibetan.DG        49.187   5.33819e-10      10       2
popdifference:                Selkup                    Tu       137.435   9.99257e-29      10      10
popdifference:                Selkup               Tubalar        86.736 

## Running Admixture F3

Preparing the population file:

In [39]:
WDIR=~/work/share/solutions
mkdir -p $WDIR
cd $WDIR
cat <<EOF > $WDIR/f3.poplist.txt
Nganasan French Finnish 
Nganasan Icelandic Finnish 
Nganasan Lithuanian Finnish 
Nganasan Norwegian Finnish 
BolshoyOleniOstrov French Finnish 
BolshoyOleniOstrov Icelandic Finnish 
BolshoyOleniOstrov Lithuanian Finnish 
BolshoyOleniOstrov Norwegian Finnish
EOF

In [41]:
cat $WDIR/f3.poplist.txt

Nganasan French Finnish 
Nganasan Icelandic Finnish 
Nganasan Lithuanian Finnish 
Nganasan Norwegian Finnish 
BolshoyOleniOstrov French Finnish 
BolshoyOleniOstrov Icelandic Finnish 
BolshoyOleniOstrov Lithuanian Finnish 
BolshoyOleniOstrov Norwegian Finnish


In [42]:
WDIR=~/work/share/solutions
mkdir -p $WDIR
cd $WDIR
cat <<EOF > f3stats.params.txt
genotypename: $SDIR/genotype_data/genotypes.smaller.geno
snpname: $SDIR/genotype_data/genotypes.smaller.snp
indivname: $SDIR/genotype_data/genotypes.smaller.ind
popfilename: $WDIR/f3.poplist.txt
inbreed: YES
EOF

In [43]:
cat f3stats.params.txt

genotypename: /home/training/work/share/genotype_data/genotypes.smaller.geno
snpname: /home/training/work/share/genotype_data/genotypes.smaller.snp
indivname: /home/training/work/share/genotype_data/genotypes.smaller.ind
popfilename: /home/training/work/share/solutions/f3.poplist.txt
inbreed: YES


In [44]:
time qp3Pop -p f3stats.params.txt

qp3Pop: parameter file: f3stats.params.txt
### THE INPUT PARAMETERS
##PARAMETER NAME: VALUE
genotypename: /home/training/work/share/genotype_data/genotypes.smaller.geno
snpname: /home/training/work/share/genotype_data/genotypes.smaller.snp
indivname: /home/training/work/share/genotype_data/genotypes.smaller.ind
popfilename: /home/training/work/share/solutions/f3.poplist.txt
inbreed: YES
## qp3Pop version: 435
inbreed set
nplist: 8
number of blocks for block jackknife: 711
snps: 593124
                      Source 1             Source 2               Target           f_3       std. err           Z    SNPs
 result:              Nganasan               French              Finnish     -0.004539       0.000510      -8.894  442567
 result:              Nganasan            Icelandic              Finnish     -0.005297       0.000563      -9.404  427954
 result:              Nganasan           Lithuanian              Finnish     -0.005062       0.000590      -8.574  426231
 result:              

## Running D stats

In [17]:
WDIR=~/work/share/solutions
cat <<EOF > $WDIR/dstat.poplist.txt
Mbuti Nganasan French Finnish 
Mbuti Nganasan Icelandic Finnish 
Mbuti Nganasan Lithuanian Finnish 
Mbuti Nganasan Norwegian Finnish 
Mbuti BolshoyOleniOstrov French Finnish 
Mbuti BolshoyOleniOstrov Icelandic Finnish 
Mbuti BolshoyOleniOstrov Lithuanian Finnish 
Mbuti BolshoyOleniOstrov Norwegian Finnish
EOF

In [18]:
cat $WDIR/dstat.poplist.txt

Mbuti Nganasan French Finnish 
Mbuti Nganasan Icelandic Finnish 
Mbuti Nganasan Lithuanian Finnish 
Mbuti Nganasan Norwegian Finnish 
Mbuti BolshoyOleniOstrov French Finnish 
Mbuti BolshoyOleniOstrov Icelandic Finnish 
Mbuti BolshoyOleniOstrov Lithuanian Finnish 
Mbuti BolshoyOleniOstrov Norwegian Finnish


In [19]:
cat <<EOF > $WDIR/dstats.params.txt
genotypename: $SDIR/genotype_data/genotypes.geno
snpname: $SDIR/genotype_data/genotypes.snp
indivname: $SDIR/genotype_data/genotypes.ind
popfilename: $WDIR/dstat.poplist.txt
f4mode: YES
EOF

In [20]:
cat $WDIR/dstats.params.txt

genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
popfilename: /home/training/work/share/solutions/dstat.poplist.txt
f4mode: YES


In [21]:
time qpDstat -p $WDIR/dstats.params.txt

qpDstat: parameter file: /home/training/work/share/solutions/dstats.params.txt
### THE INPUT PARAMETERS
##PARAMETER NAME: VALUE
genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
popfilename: /home/training/work/share/solutions/dstat.poplist.txt
f4mode: YES
## qpDstat version: 755
number of quadruples 8
  0                Mbuti   10
  1             Nganasan   11
  2   BolshoyOleniOstrov    6
  3               French   32
  4            Icelandic   12
  5           Lithuanian   10
  6            Norwegian   11
  7              Finnish    8
jackknife block size:     0.050
snps: 593124  indivs: 100
number of blocks for jackknife: 711
nrows, ncols: 100 593124
result:      Mbuti   Nganasan     French    Finnish      0.002363     19.016   29254  27852 593124 
result:      Mbuti   Nganasan  Icelandic    Finnish      0.001721     11.926   28915  27894 593124 
resul

## Outgroup F3 stats

In [27]:
cat <<EOF > $WDIR/outgroupf3stats.params.txt
genotypename: $SDIR/genotype_data/genotypes.geno
snpname: $SDIR/genotype_data/genotypes.snp
indivname: $SDIR/genotype_data/genotypes.ind
popfilename: $WDIR/outgroupF3pops_Han.txt
EOF

In [28]:
cat $WDIR/outgroupf3stats.params.txt

genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
popfilename: /home/training/work/share/solutions/outgroupF3pops_Han.txt


In [29]:
time qp3Pop -p $WDIR/outgroupf3stats.params.txt

qp3Pop: parameter file: /home/training/work/share/solutions/outgroupf3stats.params.txt
### THE INPUT PARAMETERS
##PARAMETER NAME: VALUE
genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
popfilename: /home/training/work/share/solutions/outgroupF3pops_Han.txt
## qp3Pop version: 435
nplist: 32
number of blocks for block jackknife: 711
snps: 593124
                      Source 1             Source 2               Target           f_3       std. err           Z    SNPs
 result:                   Han              Chuvash                Mbuti      0.233652       0.002072     112.782  502678
 result:                   Han             Albanian                Mbuti      0.215629       0.002029     106.291  501734
 result:                   Han             Armenian                Mbuti      0.213724       0.001963     108.882  504370
 result:                   Han   

In [30]:
cat <<EOF > $WDIR/outgroupf3stats.params.txt
genotypename: $SDIR/genotype_data/genotypes.geno
snpname: $SDIR/genotype_data/genotypes.snp
indivname: $SDIR/genotype_data/genotypes.ind
popfilename: $WDIR/outgroupF3pops_MA1.txt
EOF

In [31]:
cat $WDIR/outgroupf3stats.params.txt

genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
popfilename: /home/training/work/share/solutions/outgroupF3pops_MA1.txt


In [32]:
time qp3Pop -p $WDIR/outgroupf3stats.params.txt

qp3Pop: parameter file: /home/training/work/share/solutions/outgroupf3stats.params.txt
### THE INPUT PARAMETERS
##PARAMETER NAME: VALUE
genotypename: /home/training/share/genotype_data/genotypes.geno
snpname: /home/training/share/genotype_data/genotypes.snp
indivname: /home/training/share/genotype_data/genotypes.ind
popfilename: /home/training/work/share/solutions/outgroupF3pops_MA1.txt
## qp3Pop version: 435
nplist: 32
number of blocks for block jackknife: 711
snps: 593124
                      Source 1             Source 2               Target           f_3       std. err           Z    SNPs
 result:             MA1_HG.SG              Chuvash                Mbuti      0.243818       0.002349     103.781  350484
 result:             MA1_HG.SG             Albanian                Mbuti      0.236494       0.002296     103.008  344332
 result:             MA1_HG.SG             Armenian                Mbuti      0.231399       0.002264     102.229  349612
 result:             MA1_HG.SG   